## 목표
- RNN을 사용하여 이진수 더하기 연산을 학습시킨다

## 1. Settings
### 1) Import required libraries

In [1249]:
import copy, numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

## 2) Hyperparameter

In [1410]:
num_epochs = 10000
hidden_size = 16
batch_size = 1
num_layers = 1
lr = 0.01
output_size = 2
input_size = 2
seq_len = 8

## Create binary lookup

In [1411]:
max_binary_dim = 8
largest_number =pow(2,max_binary_dim)

In [1412]:
binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [1413]:
## Create binary lookup table - 편의상 만든거
print(np.unpackbits(np.array([8], dtype = np.uint8)))
print("===================")
# 이진수로 만들 정수 값들 1~256을 list in list 형태로 만듬.
binary = np.unpackbits(np.array([range(largest_number)], dtype=np.uint8).T, axis = 1)
print(binary.shape, binary)
print("===================")
int2binary = {}
for i in range(largest_number):
    int2binary[i] = binary[i]
print("lookup table test")
print(binary[3],int2binary[3])

[0 0 0 0 1 0 0 0]
(256, 8) [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 ...
 [1 1 1 ... 1 0 1]
 [1 1 1 ... 1 1 0]
 [1 1 1 ... 1 1 1]]
lookup table test
[0 0 0 0 0 0 1 1] [0 0 0 0 0 0 1 1]


## Make dataset

In [1414]:
def random_training_set():
    a_int = np.random.randint(1,largest_number//2)
    a  = int2binary[a_int]
    b_int = np.random.randint(1,largest_number//2) ## 두 수를 더해서 256을 넘어가면 안된다.
    b = int2binary[b_int]
    # 실제 정답 계산
    c_int = a_int + b_int
    c = int2binary[c_int]
    return a_int, b_int, Variable(torch.Tensor(np.array([a,b]))), Variable(torch.LongTensor(np.array([c])))

## 3. Model & Optimizer
### 1) Model

In [1415]:
class RNN(nn.Module):
    def __init__(self,input_size, hidden_size, output_size,num_layers):
        super(RNN,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
   
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self,input, hidden, cell):
         
        out = input.view(1,1,-1)    # out : [seq_len, batch_size, input_size]
        out, (hidden, cell) = self.rnn(out,(hidden,cell)) # out : [seq_len(=1), batch_size(=1),hidden_size(=20)]
        out = self.decoder(out.view(1,-1)) # out : [seq_len, batch_size, output_size]

        return out, hidden, cell
        
    def init_hidden(self,batch_size):
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))
        
        return hidden, cell
    
    
model = RNN(input_size, hidden_size, output_size, num_layers)

### 2) Loss & Optimizer

In [1416]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

### 3) Test Function

In [1417]:
binary2int([0,1,1,1,0,0,0,1])

113

In [1418]:
def binary2int(binary_list):
    value =0
    for i in range(7,-1,-1):
        value += binary_list[i] * pow(2,7-i)
    return value

In [1419]:
def test(predicted):
    predicted_int = binary2int(predicted)
    print("Loss :",loss,"\n","PRED: ",predicted,"\n","True: ",true,"\n",a_int, " + ", b_int," = ", predicted_int,"\n","accuracy = ",accuracy/10)

## 4. Train

In [1420]:
accuracy = 0
for i in range(100000):
    hidden,cell = model.init_hidden(batch_size)
    loss = 0
    optimizer.zero_grad()
    a_int, b_int,inp,target = random_training_set()
    predicted = []

    for j in range(7,-1,-1):
        x = inp[:,j]
        y_ = target[:,j]
        y,hidden,cell = model(x,hidden,cell)
        val,idx = y.max(1)
        loss += criterion(y,y_)
        predicted.append(int(idx[0]))
        predicted = list(reversed(predicted))
        
    true = [int(x) for x in target[0]]
    if predicted == true:
        accuracy +=1

    loss.backward()
    optimizer.step()
    
    if i % 1000 == 0:
        test(predicted)
        accuracy = 0


Loss : tensor(5.3696, grad_fn=<AddBackward0>) 
 PRED:  [0, 0, 0, 0, 0, 0, 0, 0] 
 True:  [1, 1, 0, 0, 0, 0, 0, 0] 
 103  +  89  =  0 
 accuracy =  0.0
Loss : tensor(0.0560, grad_fn=<AddBackward0>) 
 PRED:  [0, 1, 0, 1, 0, 0, 1, 1] 
 True:  [0, 1, 1, 1, 0, 0, 1, 0] 
 28  +  86  =  83 
 accuracy =  2.9
Loss : tensor(0.0062, grad_fn=<AddBackward0>) 
 PRED:  [0, 1, 0, 1, 1, 1, 0, 0] 
 True:  [0, 0, 1, 0, 0, 1, 1, 1] 
 14  +  25  =  92 
 accuracy =  7.3
Loss : tensor(0.0017, grad_fn=<AddBackward0>) 
 PRED:  [0, 1, 0, 1, 0, 1, 0, 1] 
 True:  [0, 1, 1, 0, 0, 1, 1, 0] 
 65  +  37  =  85 
 accuracy =  7.4
Loss : tensor(0.0019, grad_fn=<AddBackward0>) 
 PRED:  [1, 0, 1, 0, 1, 0, 0, 0] 
 True:  [1, 0, 0, 0, 1, 0, 0, 1] 
 124  +  13  =  168 
 accuracy =  6.7
Loss : tensor(0.0003, grad_fn=<AddBackward0>) 
 PRED:  [0, 1, 0, 0, 1, 0, 1, 1] 
 True:  [0, 1, 1, 1, 0, 0, 0, 1] 
 16  +  97  =  75 
 accuracy =  5.1
Loss : tensor(0.0002, grad_fn=<AddBackward0>) 
 PRED:  [0, 0, 0, 1, 1, 0, 1, 1] 
 True:  [0,

KeyboardInterrupt: 

In [ ]:
for j in range(7,-1,-1):
    print(a[j])

In [ ]:
a = [1,2,3,4,5,6,7,8]

In [ ]:
def test(predicted):
    print("Loss :",loss,"\n","PRED: ",predicted,"\n","True: ",true,"\n",a_int, " + ", b_int," = ", binary2int(predicted))

In [ ]:
val

In [ ]:
predicted